# Plot local color histogram differences


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import yaml

In [ ]:
# Read the CSV results
video_name = "FH" + "303_01"
# results_folder = "./results/hummingbird/color_histogram"

results_folder = "./results/wasserstein/"

df_change = pd.read_csv(
    f"./{results_folder}/{video_name}_processed_chist_diff.csv"
)

df_change.index = df_change["center_idx"]
df_change = df_change.drop(columns=["center_idx"])

# read the yaml config as well
with open(f"./{results_folder}/{video_name}_chist_config.yaml", "r") as file:
    config = yaml.safe_load(file)


# print config nicely
for key, value in config.items():
    if isinstance(value, dict):
        print(f"{key}:")
        for sub_key, sub_value in value.items():
            print(f"  {sub_key}: {sub_value}")
    else:
        print(f"{key}: {value}") 


extension = config["video_processing"]["extension"]
filename = config["video_processing"]["video_path"] #f"./data/external_data/{video_name}.{extension}"

In [ ]:
if 0:
    # Plotting the results for all columns in the dataframe starting with "std_diff_" in the same axes
    ax = plt.figure(figsize=(12, 6)).add_subplot(111)
    # for column in df_change.columns:
        # plt.plot(df_change.index, df_change[column], label=column, alpha=0.7)

    plt.plot(df_change.index, df_change["stdev_magn_diff_chist"].diff().abs(), label="diff abs", color='blue', alpha=0.7)
    plt.plot(df_change.index, df_change["stdev_magn_diff_chist"], label="mag patch", color='red', alpha=0.7)

    for x in range(1, df_change.index.max(), 500):
        ax.axvline(x=x, color='gray', linestyle='--', linewidth=0.5, zorder=0)
        ax.grid(axis='y', linestyle='--', linewidth=0.5, color='gray', zorder=0)

    plt.title("stdev_magn_diff_chist")
    plt.xlabel("Frame Index")
    plt.ylabel("Standard Deviation")
    plt.legend()
    plt.grid()
    plt.tight_layout()


        

In [ ]:
# Load ground truth CSV and extract FH401_01 rows
gt = pd.read_csv('./data/cleaned_ground_truth.csv')
gt_video = gt[gt['Video'] == video_name]
gt_video = gt_video.set_index('Frame', drop=False)

# deduplicate the index of ground truth
gt_video = gt_video[~gt_video.index.duplicated(keep='first')]
# gt_video[['Frame', 'Truth']]
print(f"number of labels in ground truth: \n {gt_video.groupby('Truth').size()}")


In [ ]:
# 0 - 1 norm
df_change['aggregated_diff'] = (df_change['stdev_magn_diff_chist'] - df_change['stdev_magn_diff_chist'].min()) / (df_change['stdev_magn_diff_chist'].max() - df_change['stdev_magn_diff_chist'].min())
# df_change['aggregated_diff'] = df_change['stdev_magn_diff_chist']

fig, ax = plt.subplots(figsize=(20, 2))
ax.plot(df_change.index, df_change['aggregated_diff'], label='aggregated_diff', color='orange', alpha=0.7)
# ax.plot(df_change.index, df_change['aggregated_diff'].diff().abs(), label='aggregated_diff.diff.abs', color='green')

for x in range(1, df_change.index.max(), 500):
    ax.axvline(x=x, color='gray', linestyle='--', linewidth=0.5, zorder=0)
    ax.grid(axis='y', linestyle='--', linewidth=0.5, color='gray', zorder=0)

# Only plot if ground truth is available
if not gt_video.empty:
    # Arrow up for Positive, arrow down for Negative
    positives = gt_video[gt_video['Truth'].str.lower() == 'positive']
    negatives = gt_video[gt_video['Truth'].str.lower() == 'negative']

    ax.scatter(positives.index, [1.01] * len(positives), marker='v', color='red', label='Positive (GT)', zorder=5)
    ax.scatter(negatives.index, [-0.05] * len(negatives), marker='^', color='blue', label='Negative (GT)', zorder=5)
    
    # Annotate positive markers with frame IDs, spreading out text if labels are close
    # last_frame_id = None
    # c = 0
    # for frame_id in positives.index:
    #     if last_frame_id is not None and abs(frame_id - last_frame_id) < 100:  # Adjust threshold as needed
    #         offset = 1.1 + 0.25 * c  # Add fixed offset of 1 if condition is true
    #         ax.text(frame_id, offset, str(frame_id), color='red', fontsize=8, ha='center')  # Spread out
    #         c += 1
    #     else:
    #         c = 0
    #         ax.text(frame_id, 1.3, str(frame_id), color='red', fontsize=8, ha='center')
    #     last_frame_id = frame_id

ax.set_ylabel('aggregated_diff')
ax.set_xlabel('center_idx')
ax.legend()

plt.title('Aggregated Diff with Ground Truth Arrows')
plt.tight_layout()
plt.show()

In [ ]:
top_k = 100
if top_k > 1:
    top_k_frames = df_change.nlargest(top_k, 'aggregated_diff')
else:
    top_k_frames = df_change[df_change['aggregated_diff'] > top_k]
    
top_k_frames = top_k_frames.sort_values(by='aggregated_diff', ascending=False)

chunk_size = 5
top_k_frames = top_k_frames.iloc[:len(top_k_frames) - len(top_k_frames) % chunk_size]

print(top_k_frames[['aggregated_diff']])



In [ ]:
import numpy as np
import random
from torchcodec.decoders import VideoDecoder

video_reader = VideoDecoder(filename)


# Set the number of random frames to sample
num_frames = 100

# define the crop box (left, top, right, bottom)
crop_box = config["preprocessing"]["crop_box"]

if 0:
    # Generate random frame indices
    random_indices = random.sample(range(len(video_reader)), num_frames)

    # Initialize an array to accumulate pixel values
    average_frame = None

    # Iterate through the random frame indices and accumulate pixel values
    for idx in random_indices:
        frame = video_reader[idx].permute(1, 2, 0).numpy()  # Convert frame to numpy array (HWC format)
        if average_frame is None:
            average_frame = np.zeros_like(frame, dtype=np.float64)
        average_frame += frame

    # Compute the average frame
    average_frame /= num_frames

    # Convert the average frame back to uint8 for visualization
    average_frame = average_frame.astype(np.uint8)

    # Display the average frame
    plt.imshow(average_frame)
    plt.title("Average Frame")
    plt.axis("off")
    plt.show()


In [ ]:
chunk_size = 5  # Number of frames per group
chunks = [top_k_frames.index[i:i + chunk_size] for i in range(0, len(top_k_frames), chunk_size)]

for chunk_idx, chunk in enumerate(chunks[:10]):
    fig, axes = plt.subplots(1, len(chunk), figsize=(15, 7))
    for i, frame_idx in enumerate(chunk):

        frame = video_reader[frame_idx]
        # frame_before = average_frame # 
        # frame_before = video_reader[frame_idx - 1]

        # Convert the frame from torch tensor to numpy array for plotting
        frame_rgb = frame.permute(1, 2, 0) #.numpy().astype(np.float64)  # Assuming frame is in CHW format
        
        # Plot the frame
        axes[i].imshow(frame_rgb)
        if crop_box:
            # Show the borders of crop, if crop_box is defined
            x, y, w, h = crop_box
            rect = plt.Rectangle((x, y), w, h, linewidth=2, edgecolor='red', facecolor='none')
            axes[i].add_patch(rect)
            
        axes[i].set_title(f"Frame {frame_idx}, score: {top_k_frames.loc[frame_idx, 'aggregated_diff']:.4f}")
        axes[i].axis("off")
    
    plt.tight_layout()
    plt.show()


In [ ]:
from video_frame_diff_colorhist import preprocess_frame, compute_patch_histograms, plot_original_triplet, visualize_histogram_difference_patch_image, compute_histogram_differences_jit

frame_skip = config["video_processing"]["frame_skip"]  # Number of frames to skip for comparison
patch_size = config["histogram_analysis"]["patch_size"]["coordinates"]  # Size of the patches for histogram analysis
bins = config["histogram_analysis"]["bins"]  # Number of bins for histogram analysis 
threshold = config["histogram_analysis"]["threshold"] # Threshold for histogram difference normalization
threshold = threshold if threshold is not None else 3 * bins # Ensure threshold is positive

# Read a specific frame from the video

for frame_index in top_k_frames.index[:10]:
    # frame_index = top_k_frames.index[0]  # Use the first frame from the top_k_frames
    # frame_index = positives.index[0] - 1  # Use the first frame from the top_k_frames

    # Buffer for preprocessed frames and histograms
    frame_buffer = []
    hist_buffer = []

    for i in [frame_index-frame_skip, frame_index, frame_index + frame_skip]:
        frame = video_reader[i].permute(1, 2, 0).cpu().numpy()
        pframe = preprocess_frame(frame, crop_box=crop_box)
        hist = compute_patch_histograms(pframe, patch_size=patch_size, bins=bins)
        frame_buffer.append(pframe)
        hist_buffer.append(hist)

    # Use buffer for computation
    hist1 = hist_buffer[0] # / hist1.sum(axis=1, keepdims=True)
    hist2 = hist_buffer[1] # / hist2.sum(axis=1, keepdims=True)
    hist3 = hist_buffer[2] # / hist3.sum(axis=1, keepdims=True)
    
    pframe1 = frame_buffer[0]
    pframe2 = frame_buffer[1]
    pframe3 = frame_buffer[2]

    
    hist_diff = compute_histogram_differences_jit(hist1, hist2, hist3, distance_metric=config["histogram_analysis"]["distance_metric"])
    
    # hist_diff = np.linalg.norm(hist2 - hist1, axis=1) + np.linalg.norm(
    #     hist3 - hist2, axis=1
    # )

    normalized_diff = (hist_diff - np.min(hist_diff)) / (np.max(hist_diff) - np.min(hist_diff)) 
                       # np.minimum(hist_diff, threshold) / threshold
    # std_dev = np.std(normalized_diff)

    plot_original_triplet(pframe1, pframe2, pframe3)
    visualize_histogram_difference_patch_image(
        pframe1, pframe2, normalized_diff, patch_size, viz_threshold=0.25,
    )


In [ ]:
# Create a DataFrame to store the results
results = []
buff = 1

sorted_scores = df_change.sort_values(by='aggregated_diff', ascending=False)

# Iterate through all top_10_frames
for frame in sorted_scores.index:
    is_within_range = any((positives['Frame'] >= frame - buff) & (positives['Frame'] <= frame + buff))
    results.append({'Frame': frame, f'within_{buff}_positive': is_within_range})

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

# print(results_df.head(20))

# Create a DataFrame to store precision and recall values
precision_recall_data = []

for k in list(range(10,201,10)) + list(range(200, 1001, 100)) + list(range(1000, 2001, 500)):
    precision_at_k = results_df[f'within_{buff}_positive'].iloc[:k].sum() / k
    true_positives = results_df["Frame"].iloc[:k].isin(positives["Frame"])
    recall_at_k = true_positives.sum() / len(positives)
    
    precision_recall_data.append({'k': k, 'Precision': precision_at_k, 'Recall': recall_at_k})

# Convert the data to a DataFrame
precision_recall_df = pd.DataFrame(precision_recall_data)

# Plot precision and recall curves with log x-axis
plt.figure(figsize=(10, 6))
plt.plot(precision_recall_df['k'], precision_recall_df['Precision'], label='Precision', marker='o')
plt.plot(precision_recall_df['k'], precision_recall_df['Recall'], label='Recall', marker='o')
plt.axvline(x=len(positives), color='red', linestyle='--', label='Actual Positives')
# plt.xscale('log')  # Set x-axis to logarithmic scale
plt.title(f'Precision and Recall Curves, actual positives: {len(positives)}')
plt.xlabel('Top-k')
plt.ylabel('Value')
plt.legend()
plt.grid()
plt.tight_layout()
plt.show()

# Plot precision-recall curve
plt.figure(figsize=(10, 6))
plt.plot(precision_recall_df['Recall'], precision_recall_df['Precision'], label='Precision-Recall Curve', marker='o')
plt.title('Precision-Recall Curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.legend()
plt.grid()
plt.tight_layout()
plt.show()

# print(precision_recall_df)